<a href="https://colab.research.google.com/github/ShashankKamath/AutoML/blob/master/Bayesian_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torchvision

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

import matplotlib.pyplot as plt

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchsummary  import summary
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.nn import Linear
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

!pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.0.1-cp36-none-any.whl size=10032 sha256=1ba08bc4d36949fc54c038acaa0a48d7ec4f313f896a76fc1892ffada85eaf91
  Stored in directory: /root/.cache/pip/wheels/1d/0d/3b/6b9d4477a34b3905f246ff4e7acf6aafd4cc9b77d473629b77
Successfully built bayesian-optimization


In [0]:
n_epochs = 3
learning_rate = 0.01
momentum = 0.5
log_interval = 10

In [0]:
batch_size_train = 64
batch_size_test = 1000

train_ds= torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,)) ]))

test_ds=torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))  ]))

train_loader = DataLoader(train_ds,batch_size=batch_size_train, shuffle=True)
test_loader = DataLoader(test_ds,batch_size=batch_size_test, shuffle=True)

0it [00:00, ?it/s]

9920512it [00:01, 9093328.41it/s]                            


Extracting /files/MNIST/raw/train-images-idx3-ubyte.gz


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 137452.56it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /files/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 2324308.74it/s]                           
0it [00:00, ?it/s]

Extracting /files/MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 53388.78it/s]            


Extracting /files/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [0]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(7 * 7 * 64, 1000)
        self.fc2 = nn.Linear(1000, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out
model=ConvNet()

In [0]:
# Define function to check model accuracy
def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

In [0]:
def get_model_accuracy():
  tot_acc = 0
  avg_acc = 0
  
  # Set model to evaluation mode
  model.eval()

  for xbt, ybt in test_loader:

    pred = model(xbt.to(device))
    tot_acc += accuracy(pred,ybt.to(device))

  avg_acc = tot_acc / len(test_loader)
  
  return avg_acc.item()


# Print accuracy of model
print("The average accuracy is: " + str(get_model_accuracy()))

The average accuracy is: 0.9795999526977539


In [0]:
class CNNtrain(nn.Module):
    def __init__(self,numLayers,channel):
        super(CNNtrain, self).__init__()
        self.layers = nn.ModuleList()
        i=0
        self.layers.append(nn.Conv2d(channel[i], channel[i+1], kernel_size=3,padding=0))
        i+=1
        for j in range(numLayers - 2):
          self.layers.append(nn.Conv2d(in_channels=channel[i], out_channels=channel[i+1], kernel_size=3,padding=0))
          i+=1
        self.layers.append(nn.Conv2d(channel[i], channel[i+1], kernel_size=3,padding=0))
        i+=2
        self.layers.append(nn.Linear(channel[i],channel[i+1]))
        i+=1
        self.layers.append(nn.Linear(channel[i],channel[i+1]))
#         print(self)

    def forward(self, x):
      y = x
#       print(y.shape)
      y = F.relu(F.max_pool2d(self.layers[0](y),kernel_size=(2, 2)))   
#       print(y.shape)
      for i in range(len(self.layers)-4):
        y = F.relu(F.max_pool2d(self.layers[i+1](y),kernel_size=(2, 2))) #self.layers[i+1](y)
#         print(y.shape)
      y = F.relu(F.max_pool2d(self.layers[-3](y),kernel_size=(2, 2)))
#       print(y.size())
      y = y.view(-1,channel[-3])
#       print("BFC",y.size())
      y = F.relu(self.layers[-2](y))
#       print("FC1",y.size())
      y = self.layers[-1](y)
#       print("FC2",y.size())
      
      return F.log_softmax(y)

In [0]:
import random

class CNNpossible(nn.Module):
    def __init__(self,numLayers):
        super(CNNpossible, self).__init__()
        self.layers = nn.ModuleList()
        channel.append(1)
        channel.append(random.randint(5,10))
        self.layers.append(nn.Conv2d(channel[-2], channel[-1], kernel_size=3,padding=0))
        for i in range(numLayers - 2):
          channel.append(random.randint(5,10))
          self.layers.append(nn.Conv2d(in_channels=channel[-2], out_channels=channel[-1], kernel_size=3,padding=0))  
        channel.append(1)
        self.layers.append(nn.Conv2d(channel[-2], channel[-1], kernel_size=3,padding=0))
        channel.append(25)
        channel.append(50)
        channel.append(10)
        self.layers.append(nn.Linear(channel[-3],channel[-2]))
        self.layers.append(nn.Linear(channel[-2],channel[-1]))
#         print("Channel Details:",channel)
#         print(self)

    def forward(self, x):
      y = x
#       print(y.shape)
      try:
        y = F.relu(F.max_pool2d(self.layers[0](y),kernel_size=(2, 2)))   
#         print("First Layer",y.shape)
        for i in range(len(self.layers)-4):
          y = F.relu(F.max_pool2d(self.layers[i+1](y),kernel_size=(2, 2))) #self.layers[i+1](y)
#           print("Middle Layer",y.shape)
        y = F.relu(F.max_pool2d(self.layers[-3](y),kernel_size=(2,2)))
#         print("Last Layer",y.size())

        y = y.view(-1,(y.size(1)*y.size(2)*y.size(3)))
#         print("Before FC",y.size())

        y = F.relu(self.layers[-2](y))
#         print("FC1",y.size())
      
        y = self.layers[-1](y)
#         print("FC2",y.size())
        
        return True
      
      except Exception as e:
#         print("Entered Exception",str(e))  
        if "Kernel size can't be greater than actual input size" in str(e):
#           print("Kernel Size exceed- Not possible config")
          return False #torch.tensor([0])
        
        
        elif "is invalid for input of size" or "size mismatch" in str(e):
#           print("Fully Connected Change required")
#           y = y.view(-1,(y.size(1)*y.size(2)*y.size(3)))
          y = y.view(y.size(0),-1)
          channel[-3]=y.size(1)
          channel[-2]=random.randint(5,50)
#           print("Channel Details after FC:",channel)
          return True



In [0]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [0]:
def train(model2,optimizer2,epoch):
  model2.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer2.zero_grad()
    output = model2(data)
    loss = F.nll_loss(output, target)  
    loss.backward()
    optimizer2.step()
    if batch_idx % log_interval == 0:
#       print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#         epoch, batch_idx * len(data), len(train_loader.dataset),
#         100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append((batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
      torch.save(model2.state_dict(), 'model2.pth')
      torch.save(optimizer2.state_dict(), 'optimizer2.pth')

In [0]:
def test(model1):
  model1.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = model1(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  accuracy= 100. * correct / len(test_loader.dataset)
#   print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
#     test_loss, correct, len(test_loader.dataset),
#     100. * correct / len(test_loader.dataset)))
  return accuracy

In [0]:
def test_possible(model):
    for data, target in test_loader:
      result = model(data)
      if result==False:
#         print("Correct Layer:",num_layer)
#         print("Channel: ",channel)
        y.append([0])
        x.append([num_layer,channel])
#         print("X",x)
#         print("Y",y)
        break
      elif result==True:
#         print("Correct Layer:",num_layer)
#         print("Channel: ",channel)  
        x.append([num_layer,channel])
        model2 = CNNtrain(num_layer,channel)
        optimizer2 = optim.SGD(model2.parameters(), lr=learning_rate,momentum=momentum)  
#         accuracy=test(model2)
#         print("Accuracy Before Training:",accuracy)
        for epoch in range(1, n_epochs + 1):
          train(model2,optimizer2,epoch)
        accuracy=int(test(model2))
        print("Accuracy after Training:",accuracy)
        y.append([accuracy])
#         print("X",x)
#         print("Y",y)         
        break
                 
x=[]
y=[]
n_epochs=5
for run in range(1,2):
  num_layer=random.randint(1,5)
  channel=[]
  model = CNNpossible(num_layer)
  optimizer = optim.SGD(model.parameters(), lr=learning_rate,momentum=momentum)
  test_possible(model)

print(x)
print(y)

Bayesian Optimization

In [0]:
def obj_func(lr, bs, epochs , optimizervalue, loss):
      
      # We need to round off bs and epochs because Gaussian processes cannot deal with discrete variables 
      bs = int(bs)
      # print("Batch Size",bs)
      epochs = int(epochs)
      # print("Number of epochs",epochs)
      optimizervalue=int(round(optimizervalue))
      # print("Optimizer Value",optimizervalue)
      loss=int(round(loss))
      # print("Loss Function Value",loss)

      train_loader = DataLoader(train_ds,batch_size=bs, shuffle=True)
      test_loader = DataLoader(test_ds,batch_size=bs, shuffle=True)
      
      
      if optimizervalue==1:
          optimization = optim.Adam(model.parameters(), lr = lr)
          print("Adam Optimizer")
      elif optimizervalue==2:
          optimization = optim.SGD(model.parameters(),lr=lr)
          print("SGD Optimizer")
      elif optimizervalue==3:
          optimization = optim.RMSprop(model.parameters(),lr=lr)
          print("RMSProp Optimizer")

      if loss==1:
        loss_func = F.cross_entropy
        print("Cross Entropy Loss")
      elif loss==2:
        loss_func = F.nll_loss
        print("NLL Loss")
      elif loss==3:
        loss_func = F.mse_loss
        print("MSE Loss")


      #  loss MSELoss,CrossEntropyLoss, NLLLoss
      for epoch in range(epochs):
    
        for xb, yb in train_loader:
        
            # .to(device) moves torch.Tensor objects to the GPU for faster training
        
            preds = model(xb.to(device))
            loss = loss_func(preds, yb.to(device))
            acc = accuracy(preds,yb.to(device))
        
            loss.backward()
            optimization.step()
            optimization.zero_grad()
        
        print("Loss: " + str(loss.item()) + "\t \t Accuracy: " + str(100 * acc.item()))

      acc = get_model_accuracy()
      
      return acc

In [0]:
from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction

utility = UtilityFunction(kind="ucb", kappa=2.5, xi=0.0)

# Bounded region of parameter space
pbounds = {'lr': (1e-4, 1e-2), 
           'bs': (64, 512), 
           'epochs': (1,3), 
           'optimizervalue':(1,3),
           'loss':(1,2) }
          #  'layers':(1,4), 
          #  'units':(1,10)}

model = ConvNet()

optimizer = BayesianOptimization(
    f=obj_func,
    pbounds=pbounds,
    verbose=2, # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1,
)


# for _ in range(5):
#     next_point = optimizer.suggest(utility)
#     target = obj_func(**next_point)
#     optimizer.register(params=next_point, target=target)
#     print(target, next_point)
# print(optimizer.max)

optimizer.maximize(init_points=2, n_iter=3,)
print(optimizer.max)


for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

|   iter    |  target   |    bs     |  epochs   |   loss    |    lr     | optimi... |
-------------------------------------------------------------------------------------
Adam Optimizer
Cross Entropy Loss
Loss: 0.17216931283473969	 	 Accuracy: 95.59999704360962
Loss: 0.20090298354625702	 	 Accuracy: 95.20000219345093
|  1        |  0.9819   |  250.8    |  2.441    |  1.0      |  0.003093 |  1.294    |
SGD Optimizer
Cross Entropy Loss
Loss: 0.1523716151714325	 	 Accuracy: 95.55555582046509
|  2        |  0.9841   |  105.4    |  1.373    |  1.346    |  0.004028 |  2.078    |
RMSProp Optimizer
Cross Entropy Loss
Loss: 0.2841380834579468	 	 Accuracy: 93.75
Loss: 2.270498752593994	 	 Accuracy: 21.875
|  3        |  0.101    |  64.0     |  2.069    |  1.44     |  0.007613 |  2.775    |
RMSProp Optimizer
Cross Entropy Loss
Loss: 2.3266098499298096	 	 Accuracy: 11.11111119389534
|  4        |  0.1135   |  103.5    |  1.738    |  1.219    |  0.006548 |  2.966    |
SGD Optimizer
NLL Loss
Loss: 